In [ ]:
%pip install pandas

In [ ]:
# load data
import pandas as pd
pd.read_csv("PRONTOBenchmarkTest9.csv")

,Start Time,09/12/2017 10:00,End Time,9/12/2017 14:00:00 PM,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31
0,NaN,FT305/OUT.CV,FT302/OUT.CV,FT305/AI2/OUT.CV,PT312/OUT.CV,FT102/OUT.CV,FT104/OUT.CV,FT102/AI3/OUT.CV,FT102/AI2/OUT.CV,PT417/OUT.CV,...,FIC302/PID1/PV.CV,FIC301/PID1/OUT.CV,FIC301/PID1/SP.CV,FIC301/PID1/PV.CV,FIC102/PID1/OUT.CV,FIC102/PID1/SP.CV,FIC102/PID1/PV.CV,FIC101/PID1/OUT.CV,FIC101/PID1/SP.CV,FIC101/PID1/PV.CV
1,TIMESTAMP,Air In1,Air In2,Air T,Air P,Water In1,Water In2,Water T,Water Density,Mixture zone P,...,NaN,NaN,Air In 2,Air In 2,Water In1,Water In1,NaN,Water In 2,Water In 2,NaN
2,09/12/2017 10:00,99.7911377,0.002018351,16.46453094,1.157050967,0.00014997,0.008798941,18.75638962,998.2576294,1.131950021,...,99.69100189,0,100.0009995,0.002018351,0,0,0.00014997,0,0,0.00879894
3,09/12/2017 10:00,99.84359741,0.00171826,16.46552086,1.157019019,0.000150975,0.008799082,18.75638962,998.2578125,1.131800056,...,99.74504089,0,100.0009995,0.00171826,0,0,0.000150975,0,0,0.00879908
4,09/12/2017 10:00,99.84837341,0.001418168,16.46649933,1.156985998,0.000151979,0.008799221,18.51082993,998.2579956,1.131649971,...,99.79908752,0,100.0009995,0.001418168,0,0,0.000151979,0,0,0.00879922
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14398,09/12/2017 13:59,2.057109118,0.000166574,19.79092026,0.249562606,0.002618832,0.002031899,24.25476074,996.571106,0.284789801,...,2.100024939,0,100.0009995,0.000166574,0,0,0.002618832,0,0,0.002031899
14399,09/12/2017 13:59,1.971276999,0.00016606,19.79129982,0.2495296,0.002621138,0.002035049,24.25476074,996.5722046,0.284790009,...,2.061877012,0,100.0009995,0.00016606,0,0,0.002621138,0,0,0.002035049
14400,09/12/2017 13:59,1.986773968,0.000165545,19.79166985,0.249496594,0.002623444,0.0020382,24.4118309,996.5733032,0.284790307,...,2.02373004,0,100.0009995,0.000165545,0,0,0.002623444,0,0,0.0020382
14401,09/12/2017 13:59,2.002271891,0.000165031,19.79204941,0.249463603,0.002625749,0.00204135,24.4118309,996.5742798,0.284790605,...,2.016311884,0,100.0009995,0.000165031,0,0,0.002625749,0,0,0.00204135
